# Higgs Boson - ML Challenge

In [4]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from implementation import *
from model_helpers import *
from data_helpers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the dataset

In [8]:
paths = { 
    'train' : 'data/train.csv',
     'test' : 'data/test.csv',
     'submission' : 'data/sample-submission.csv'
        }

y_tr, tx_tr, ids_tr = load_csv_data(paths['train'], sub_sample=False)
y_te, tx_te, ids_te = load_csv_data(paths['test'])

len_test = len(y_te)

y_tr = y_tr[:, np.newaxis]
y_pred = np.zeros(len_test)

array([[ 1.],
       [-1.],
       [-1.],
       ...,
       [ 1.],
       [-1.],
       [-1.]])

## Cross Validation

In [ ]:
def cross_validation(y, x, k_indices, k, lambda_, degree, initial_w, max_iters, gamma):
    """return the loss of ridge regression for a fold corresponding to k_indices
    
    Args:
        y:          shape=(N,)
        x:          shape=(N,)
        k_indices:  2D array returned by build_k_indices()
        k:          scalar, the k-th fold (N.B.: not to confused with k_fold which is the fold nums)
        lambda_:    scalar, cf. ridge_regression()
        degree:     scalar, cf. build_poly()
        gamma:      scalar, stepsize

    Returns:
        train and test errors (probability of predicting correct values)

    """
    
    train_id = np.delete(k_indices, k, axis=0).ravel()
    test_id = k_indices[k]
    
    x_tr, y_tr = x[train_id], y[train_id]
    x_te, y_te = x[test_id], y[test_id]
    
    x_tr, x_te = pre_processing(x_tr, x_te, poly_deg=degree, gamma=gamma)
    
    w, _ = reg_logistic_regression(y_tr, x_tr, lambda_, initial_w=initial_w, max_iters=max_iters, gamma=gamma)
    
    return (y_tr == predict_labels(w, x_tr)).mean(), (y_te == predict_labels(w, x_te)).mean()

In [ ]:
def cross_validation_grid_search(txs, ys):
    """Runs cross validation on the data with different values of hyperparameters to compare accuracy
    
    Args:
        txs: subsets of train dataset
        ys:  labels of the different subsets
        
    """
    # Hyperparameters
    seed = 50
    k_fold = 3
    lambdas = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
    degrees = range(2, 8)
    gammas = range(0, 11)
    
    # split data in k fold
    k_indices = []
    for i in range(len(txs)):
        k_indices.append(build_k_indices(ys[i].shape[0], k_fold, seed))
        
    # cross validation
    for i in range(len(txs)):
        max_acc = 0
        print(f"*Set {i}")
        for l in lambdas:
            for d in degrees:
                for g in gammas:
                    pred_pcts = []
                    for k in range(k_fold):
                        pred_pct = cross_validation(ys[i], txs[i], k_indices[i], k, l, d, g)
                        pred_pcts.append(pred_pct)
                    pct = np.mean(pred_pcts)
                    if pct > max_acc:
                        max_acc = pct
                        print(f">>>>Set {i}/lamdba={l}/deg={d}/gamma={g}/ACC={np.around(pct, 4)}")

## Train the model with the best parameters

In [ ]:
def train_model(txs, ys, lambda_, gamma):
    """Trains the classifier model
    
    Args:
        txs: training data split into three subsets
        y: labels of training data split into three subsets
    
    Returns:
        ws: weights of each subsets.
    """
    
    ws = []
    
    for i in range(len(txs)):
        weights, loss = reg_logistic_regression(ys[i], txs[i], lambda_=lambda_, initial_w=np.zeros((txs[i].shape[1], 1)), max_iters=1000, gamma=gamma)
        ws.append(weights)
        
    return ws

## Generate submission

In [ ]:
def generate_predictions(txs_te, ws):
    """Generate the predictions and save ouput
    
    Args:
        txs_te: subsets of test dataset
        ws: weights of the different subsets
        
    """
    
    mask_test = get_jet_indexes(tx_test)
    for j in range(len(txs_test)):
            y_pred[mask_test[j]] = [i[0] for i in predict_labels(ws[j], txs_test[j])]
            
    create_csv_submission(ids_test, y_pred, paths['submission'])